In [1]:
! python3 -m ipykernel install --user --name=main-venv

Installed kernelspec main-venv in /home/roman/.local/share/jupyter/kernels/main-venv


In [2]:
import os

import json

from tqdm import tqdm

import pandas as pd
import numpy as np

from pydantic import BaseModel, Field

from typing import List

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.exceptions import OutputParserException

from langchain_gigachat.chat_models import GigaChat

from datasets.parser_dataset import ParserDataset

In [3]:
DATA_DIR = "data"

# ✅Build Giga based solution

In [4]:
class GigaOutput(BaseModel):

    topic: str = Field(description='''
        Описание темы на форуме.
    ''')
    menace_status: str = Field(description='''
        Ответ от Гигачата на вопрос, является ли данная
        тема на форуме потенциально мошеннической.
    '''),
    comment: str = Field(description='''
        Небольшой комментарий от Гигачата,
        почему он посчитал тему потенциально мошеннической.
    ''')

class GigaMapper:

    def __init__(self, token: str, prompt: str, model: str = 'GigaChat-2'):

        self.model = GigaChat(
            credentials=token,
            verify_ssl_certs=False,
            model=model
        ).with_structured_output(GigaOutput, method="json_mode")

        self.messages = [
            SystemMessage(content=prompt)
        ]

    def inference(self, topics: List[str]):

        res = []
        
        for topic in tqdm(topics):

            try:
                answer = self.model.invoke(self.messages + [HumanMessage(topic)])
            except OutputParserException:
                answer = {'topic': topic, 'menace_status': 'да', 'comment': 'чувствительная тема'}
                
            res.append(dict(answer))
        
        return res

In [5]:
with open('gc-token.txt', 'r') as file:
    TOKEN = file.read()

system_prompt = '''
    Ты - помошник по борьбе с онлайн мошенничеством в банке.
    Тебе дают заголовок темы на одном из онлайн банковских форумов.
    Твоя задача - определить, является ли тема потенциально мошеннической - 
    если да, ответь "да", иначе - "нет".
    Также дай небольшой комментарий, почему ты так решил.

    Проанализируй тему и верни ответ в формате JSON со структурой:
        {
          "topic": "оригинальное название темы"
          "menace_status": "да/нет",
          "comment": "твой короткомментарий"
        }

    Запрещено:
        - Добавлять посторонний текст
        - Менять структуру JSON
'''

giga = GigaMapper(TOKEN, system_prompt)

# ✅Map Kupus.ru topics

## Giga inference

In [6]:
dataset = ParserDataset("parser/config/kupus.json", "data/kupus_headers.json", "data/kupus_comments")
kupus_topics_df = dataset.get_headers()

In [7]:
kupus_topics_df = pd.DataFrame(giga.inference(kupus_topics_df['flud name'].values))

100%|█████████████████████████████████████████| 158/158 [01:16<00:00,  2.06it/s]


In [8]:
kupus_topics_df.to_csv(os.path.join(DATA_DIR, 'kupus_mapping_credits.csv'))

In [9]:
kupus_topics_df[kupus_topics_df['menace_status'] == 'да']

,topic,menace_status,comment
151,Мобильные приложения. Клонирование.,да,Тема указывает на потенциальную угрозу клониро...
152,С какими мошенническими схемами вы сталкивались?,да,Тема косвенно указывает на обсуждение возможны...
157,МКК Академическая и Право онлайн. Мошенническа...,да,Содержит признаки возможного мошенничества: уп...


# ✅Map Hranidengi.com topics

## Giga inference

In [10]:
dataset = ParserDataset("parser/config/hranidengi.json", "data/hranidengi_headers.json", "data/hranidengi_comments")
hranidengi_topics_df = dataset.get_headers()

In [11]:
hranidengi_topics_df = pd.DataFrame(giga.inference(hranidengi_topics_df['flud name'].values))

100%|█████████████████████████████████████████| 157/157 [01:16<00:00,  2.06it/s]


In [12]:
hranidengi_topics_df.to_csv(os.path.join(DATA_DIR, 'hranidengi_mapping_credits.csv'))

In [13]:
hranidengi_topics_df[hranidengi_topics_df['menace_status'] == 'да']

,topic,menace_status,comment
4,Мошенники через Яндекс доставку,да,Тема указывает на возможную схему мошенничеств...
5,Схематоз с кредитными картами,да,"Тема содержит слово 'схематоз', что обычно ука..."
11,Кидалово интернет-магазина з/частей с оплатой ...,да,Тема указывает на возможный случай мошенничест...
16,Банк заблокировал все карты и ИБ и отказываетс...,да,Тема указывает на возможные мошеннические дейс...
20,Банк Альфа вымогает деньги за предоставление с...,да,Тема указывает на возможные незаконные действи...
100,Вопрос ламера: как испортить КИ любимому родст...,да,чувствительная тема
104,Передача банковской карты другому лицу для пок...,да,"Потенциально мошенническая практика, так как п..."
117,"Телефонный террор ООО ПКО ""Авангард"" - как нак...",да,Тема содержит упоминание о телефонном террориз...
121,Признание мошеннического договора МТС банка не...,да,Тема содержит указание на возможный факт мошен...
124,Почему дропов не привлекают к уголовной ответс...,да,Тема содержит намек на незаконные действия и м...
